Importamos las bibliotecas necesarias

In [2]:
import pandas as pd
import json
import ast
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

Se converten los 3 datasets en dataframes, primero se desanida todo el código porque vienen en una lista en formato .json

In [2]:
#Creo una lista vacía para almacenar los datos del archivo JSON
juegos = [] 
#Abro el archivo JSON en modo lectura, y con el bucle for se lee línea por línea
with open('output_steam_games.json') as file:
    for line in file: 
        data = json.loads(line)
        #Añadimos cada objeto a la lista creada en un principio
        juegos.append(data)

#Se crea el dataframe con los datos de la lista
df_juegos = pd.DataFrame(juegos)


In [7]:
#Verificamos si hay valores nulos en el df para poder eliminarlos
#df_juegos.info()

In [3]:
#Procedemos a eliminar todos los valores nulos que tenemos en el df
#Utilizamos el método .dropna(inplace=True),
#el inplace=True es para que la eliminación se realice sobre el original y no se genere un nuevo df
df_juegos.dropna(inplace=True)

In [4]:

resenias = []
with open('australian_user_reviews.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        resenias.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(resenias)



In [5]:
items = []
with open('australian_users_items.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        items.append(ast.literal_eval(line))

df_items = pd.DataFrame(items)


Se normalizan los dataframes

- Se explotan las columnas anidadas para dividirlas en múltiples columnas y se unen al dataframe original

In [6]:
dfresenias_exploded = df_reviews.explode('reviews')
dfresenias_normalized = pd.json_normalize(dfresenias_exploded['reviews'])
dfresenias_exploded.reset_index(drop=True, inplace=True)
dfresenias_normalized.reset_index(drop=True, inplace=True)

dfresenias_final= pd.concat([dfresenias_exploded.drop('reviews', axis=1), dfresenias_normalized], axis=1)

In [7]:
df_items_exploded = df_items.explode('items')
df_items_normalized = pd.json_normalize(df_items_exploded['items'])
df_items_exploded.reset_index(drop=True, inplace=True)
df_items_normalized.reset_index(drop=True, inplace=True)

dfitems_final= pd.concat([df_items.drop('items', axis=1), df_items_normalized], axis=1)

In [8]:
juegos_exploded = df_juegos.explode('genres')
juegos_normalized = pd.json_normalize(juegos_exploded['genres'])
juegos_exploded.reset_index(drop=True, inplace=True)
juegos_normalized.reset_index(drop=True, inplace=True)

df_juegos_final = pd.concat([df_juegos, juegos_normalized], axis=1, sort=False).reset_index(drop=True)

In [15]:
print(df_juegos_final.isnull().sum())

genres          0
app_name        0
title           0
release_date    0
id              0
developer       0
dtype: int64


In [11]:
#Elimino columnas innecesarias de juegos

df_juegos_final.drop(columns=['publisher'], inplace=True)
df_juegos_final.drop(columns=['url'], inplace=True)
df_juegos_final.drop(columns=['tags'], inplace=True)
df_juegos_final.drop(columns=['price'], inplace=True)
df_juegos_final.drop(columns=['specs'], inplace=True)
df_juegos_final.drop(columns=['early_access'], inplace=True)
df_juegos_final.drop(columns=['reviews_url'], inplace=True)


In [14]:
#Elimino nulos que se encontraron 
df_juegos_final=df_juegos_final.dropna(subset=['genres'])
df_juegos_final=df_juegos_final.dropna(subset=['release_date'])

In [22]:
dfitems_final

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,js41637,20,Team Fortress Classic,0.0
2,evcentric,30,Day of Defeat,7.0
3,Riot-Punch,40,Deathmatch Classic,0.0
4,doctr,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
88305,76561198323066619,202090,Magicka: Wizard Wars,1501.0
88306,76561198326700687,239220,The Mighty Quest For Epic Loot,2374.0
88307,XxLaughingJackClown77xX,257730,Infinity Wars - Animated Trading Card Game,654.0
88308,76561198329548331,227220,Sang-Froid - Tales of Werewolves,2973.0


In [19]:
#Eliminoo columnas innecesarias de items
dfitems_final.drop(columns=['user_url'], inplace=True)
dfitems_final.drop(columns=['playtime_2weeks'], inplace=True)
dfitems_final.drop(columns=['steam_id'], inplace=True)
dfitems_final.drop(columns=['items_count'], inplace=True)


In [21]:
# Elimino nulos en items
dfitems_final=dfitems_final.dropna(subset=['user_id'])

In [28]:
print(dfresenias_final.isnull().sum())


user_id      0
posted       0
item_id      0
recommend    0
review       0
dtype: int64


In [24]:
#Elimino columnas innecesarias de resenias
dfresenias_final.drop(columns=['user_url'], inplace=True)
dfresenias_final.drop(columns=['funny'], inplace=True)
dfresenias_final.drop(columns=['last_edited'], inplace=True)
dfresenias_final.drop(columns=['helpful'], inplace=True)

In [27]:
#Elimino valores nulos de resenias
dfresenias_final=dfresenias_final.dropna(subset=['item_id'])

Ahora creo la columna de 'sentiment_analysis' en dfresenias_final, aplicando análisis de sentimiento con NLP calculando el puntaje de polaridad.

In [29]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [238]:
# Descargamos los recursos necesarios para NLTK (solo es necesario la primera vez)
#nltk.download('vader_lexicon')

In [30]:
#Se crea el analizador de sentimientos

sia= SentimentIntensityAnalyzer()

#Función para aplicar el análisis de sentimiento

def analyze_sentiment_or_default(text):
    if isinstance(text,str):
        compound_score = sia.polarity_scores(text)['compound']
        if compound_score >= 0.05:
            return 2
        elif compound_score <= -0.05:
            return 0
        else:
            return 1
    else:
        return 1

#Aplicamos el análisis sobre la columna 'review' y creamos la columna 'sentiment_analysis', que va a reemplazar a la columna review.

dfresenias_final['sentiment_analysis']= dfresenias_final['review'].apply(analyze_sentiment_or_default)
dfresenias_final.drop(columns=['review'], inplace=True)

C:\Users\Mary\AppData\Local\Temp\ipykernel_3672\1186812390.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresenias_final['sentiment_analysis']= dfresenias_final['review'].apply(analyze_sentiment_or_default)
C:\Users\Mary\AppData\Local\Temp\ipykernel_3672\1186812390.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfresenias_final.drop(columns=['review'], inplace=True)


Se guardan los datasets provisionales para hacer las funciones

In [32]:
df_juegos_final.to_csv('juegos_funciones.csv')
dfitems_final.to_csv('items_funciones.csv')
dfresenias_final.to_csv('resenias_funciones.csv')

Abrimos los archivos csv para utilizarlos

In [33]:
df_juegos = pd.read_csv('juegos_funciones.csv')
df_items = pd.read_csv('items_funciones.csv')
df_resenias = pd.read_csv('resenias_funciones.csv')

In [71]:
df_juegos.columns

Index(['genres', 'app_name', 'title', 'release_date', 'item_id', 'developer'], dtype='object')

In [72]:
df_items.columns

Index(['user_id', 'item_id', 'item_name', 'playtime_forever'], dtype='object')

In [73]:
df_resenias.columns

Index(['user_id', 'posted', 'item_id', 'recommend', 'sentiment_analysis'], dtype='object')

- Se analiza si hay valores nulos en las columnas
- Se limpian las columnas que se utilizarán para no tener valores nulos y se transforma el tipo de dato de fechas para que sea manejable
- Eliminamos columnas innecesarias

In [37]:
#Se eliminan las columnas que se agregaron con los índices al momento de crear el nuevo csv
df_resenias.drop(columns=['Unnamed: 0'], inplace=True)
df_items.drop(columns=['Unnamed: 0'], inplace=True)
df_juegos.drop(columns=['Unnamed: 0'], inplace=True)

In [239]:
#Vemos si hay valores nulos en cada dataframe
#print(df_juegos.isnull().sum()) 
#print(df_resenias.isnull().sum()) 
#print(df_items.isnull().sum()) 

In [42]:
#Elimino nulos de items
df_items = df_items.dropna(subset=['item_name'])

In [64]:
#Cambio el tipo de dato de la columna release_date
df_juegos['release_date'] = df_juegos['release_date'].apply(pd.to_datetime,format='%d/%m/%Y')

In [69]:
#Cambio de nombre la columna id para luego poder hacer el merge
df_juegos = df_juegos.rename(columns={"id": "item_id"})

In [76]:
# Se sacan los valores que tengan en platime_forever 0.0
df_items=df_items[df_items['playtime_forever'] != 0.0]

In [78]:
#Merge para funciones 1 y 2(juegos e items)

juegos_items= pd.merge(df_juegos, df_items, on='item_id')

In [79]:
#Merge para funciones 3, 4 y 5(juegos y resenias)

juegos_resenias= pd.merge(df_juegos,df_resenias,on='item_id')

In [87]:
#Eliminamos los valores nulos de release_date

#juegos_resenias = juegos_resenias.dropna(subset=['release_date'])
#juegos_items = juegos_items.dropna(subset=['release_date'])

In [89]:
#Guardo los nuevos datos definitivos para las funciones en archivos .csv

juegos_resenias.to_csv('df_f3_4_5.csv')
juegos_items.to_csv('df_f1_2.csv')

In [3]:
#Creamos df para funciones 1 y 2 del archivo df_f1_2.csv

df_f1_2= pd.read_csv('df_f1_2.csv')

In [8]:
df_f1_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48889 entries, 0 to 48888
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            48889 non-null  object 
 1   app_name          48889 non-null  object 
 2   title             48889 non-null  object 
 3   release_date      48889 non-null  object 
 4   item_id           48889 non-null  int64  
 5   developer         48889 non-null  object 
 6   user_id           48889 non-null  object 
 7   item_name         48889 non-null  object 
 8   playtime_forever  48889 non-null  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 3.4+ MB


In [4]:
df_f1_2['genres'] = df_f1_2['genres'].apply(lambda x: x.replace("'", "").strip("[]"))

In [6]:
df_f1_2.drop(columns='Unnamed: 0', inplace=True)

In [134]:
df_f1_2['release_date']= df_f1_2['release_date'].dt.year

In [133]:
df_f1_2['release_date'] = pd.to_datetime(df_f1_2['release_date'])

## Primera función
-  Indicación: def PlayTimeGenre(genero:str): Debe devolver año con mas horas jugadas para dicho género. <br> Ejemplo de retorno: {"Año con más horas jugadas para Género X" : 2013}

In [147]:
def PlayTimeGenre(genero):
    genero = genero.lower()
    genero = genero.capitalize()
    df_f1_2['release_date'] = pd.to_datetime(df_f1_2['release_date'], errors='coerce')
    max_horas_anio = None
    max_horas = 0
    horas_por_anio = {}
    
    for index, row in df_f1_2.iterrows():
        if genero in row['genres']:
            # Obtener el año de la fecha de lanzamiento
            year = row['release_date'].year
            
            # Sumar las horas jugadas
            horas_jugadas = row['playtime_forever']
            
            if year not in horas_por_anio:
                horas_por_anio[year] = 0
                
            horas_por_anio[year] += horas_jugadas
            
            if horas_por_anio[year] > max_horas:
                max_horas = horas_por_anio[year]
                max_horas_anio = year
    res = {
    "Año con más horas": max_horas_anio,"Total de horas sumadas": max_horas
    }
            
    return res

In [149]:
PlayTimeGenre('Action')

{'Año con más horas': 2012, 'Total de horas sumadas': 23939553.0}

## Segunda función
- Indicaciones:  def UserForGenre(genero:str): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.<br> Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [13]:

def UserForGenre(genero):

    df_f1_2["release_date"] = pd.to_datetime(df_f1_2["release_date"])
    df_f1_2["release_year"] = df_f1_2["release_date"].dt.year

    # Filtrar el dataframe por género.
    df_filtrado = df_f1_2[df_f1_2["genres"].str.contains(genero)]

    # Calcular la acumulación de horas jugadas por usuario.
    df_acumulado = df_filtrado.groupby("user_id")["playtime_forever"].sum()

    # Obtener el usuario con más horas jugadas.
    usuario_mas_horas = df_acumulado.idxmax()

    # Calcular la acumulación de horas jugadas por año.
    df_acumulado_por_ano_1 = df_filtrado.groupby(["release_year"])["playtime_forever"].sum().to_frame()
    df_1 = df_acumulado_por_ano_1.add_suffix("_Sum").reset_index()

    # Convertir el dataframe a una lista de diccionarios.
    df_1 = df_1.rename(columns={"release_year": "Año", "playtime_forever_Sum": "Horas"})
    lista_acumulado_por_ano = df_1.to_dict(orient="records")

    # Devolver el resultado.
    return {
        "Usuario con más horas jugadas para Género X": usuario_mas_horas,
        "Horas jugadas": lista_acumulado_por_ano
    }


In [14]:
UserForGenre('Racing')

{'Usuario con más horas jugadas para Género X': 'gossuper',
 'Horas jugadas': [{'Año': 1994, 'Horas': 21.0},
  {'Año': 1995, 'Horas': 431.0},
  {'Año': 1996, 'Horas': 5.0},
  {'Año': 1997, 'Horas': 18.0},
  {'Año': 1998, 'Horas': 87.0},
  {'Año': 2003, 'Horas': 6620.0},
  {'Año': 2005, 'Horas': 3094.0},
  {'Año': 2006, 'Horas': 6117.0},
  {'Año': 2007, 'Horas': 7550.0},
  {'Año': 2008, 'Horas': 15892.0},
  {'Año': 2009, 'Horas': 34854.0},
  {'Año': 2010, 'Horas': 76373.0},
  {'Año': 2011, 'Horas': 16011.0},
  {'Año': 2012, 'Horas': 30078.0},
  {'Año': 2013, 'Horas': 89352.0},
  {'Año': 2014, 'Horas': 147882.0},
  {'Año': 2015, 'Horas': 845215.0},
  {'Año': 2016, 'Horas': 66370.0},
  {'Año': 2017, 'Horas': 343.0}]}

Abro el archivo con el que voy a trabajar en las funciones 3, 4 y 5

In [15]:
df_f3_4_5= pd.read_csv('df_f3_4_5.csv')

In [17]:
#Elimino columnas innecesarias
df_f3_4_5.drop(columns='Unnamed: 0',inplace=True)
df_f3_4_5.drop(columns='title',inplace=True)
df_f3_4_5.drop(columns='release_date',inplace=True)


In [19]:
df_f3_4_5['genres'] = df_f3_4_5['genres'].apply(lambda x: x.replace("'", "").strip("[]"))

In [21]:
#Obtengo el año de la columna posted y se crea la columna posted_year
df_f3_4_5.dropna(inplace=True)
lista=[]
for i in range(0,len(df_f3_4_5)):
    string = df_f3_4_5.iloc[i][5]

    try:
      b = int(string[-5:-1])
    except ValueError:
      b = float('nan') 

    lista.append(b)

df_f3_4_5['posted_year'] = lista
df_f3_4_5.dropna(inplace=True)
df_f3_4_5['posted_year'] = df_f3_4_5['posted_year'].astype('int')

C:\Users\Mary\AppData\Local\Temp\ipykernel_732\1974894324.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string = df_f3_4_5.iloc[i][5]


In [23]:
#Elimino la columna posted
df_f3_4_5.drop(columns='posted', inplace=True)

## Tercera función
- def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales) <br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [25]:
def UsersRecommend(year: int):
    # Verificar si el año es igual a -1 y mostrar un mensaje personalizado
    if year == -1:
        return "El año ingresado es -1, lo cual no es válido."

    # Verificar que el año sea un número entero
    if not isinstance(year, int):
        return "El año debe ser un número entero."

    # Verificar que el año ingresado esté en la columna 'posted_year'
    if year not in df_f3_4_5['posted_year'].unique():
        return "El año no se encuentra en la columna 'posted_year'."

    # Filtrar el dataset para obtener solo las filas correspondientes al año dado
    juegos_del_año = df_f3_4_5[df_f3_4_5['posted_year'] == year]

    # Calcular la cantidad de recomendaciones para cada juego
    recomendaciones_por_juego = juegos_del_año.groupby('app_name')['recommend'].sum().reset_index()

    # Ordenar los juegos por la cantidad de recomendaciones en orden descendente
    juegos_ordenados = recomendaciones_por_juego.sort_values(by='recommend', ascending=False)

    # Tomar los tres primeros lugares
    primer_puesto = juegos_ordenados.iloc[0]['app_name']
    segundo_puesto = juegos_ordenados.iloc[1]['app_name']
    tercer_puesto = juegos_ordenados.iloc[2]['app_name']

    # Crear el diccionario con los tres primeros lugares
    top_tres = {
        "Puesto 1": primer_puesto,
        "Puesto 2": segundo_puesto,
        "Puesto 3": tercer_puesto
    }

    return top_tres

In [26]:
UsersRecommend(2015)

{'Puesto 1': 'Counter-Strike: Global Offensive',
 'Puesto 2': 'Team Fortress 2',
 'Puesto 3': "Garry's Mod"}

## Cuarta función
- def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos) <br>
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [27]:
def UsersWorstDeveloper(año: int):
    # Verificar si el año es igual a -1 y mostrar un mensaje personalizado
    if año == -1:
        return "El año ingresado es -1, lo cual no es válido."

    # Verificar que el año sea un número entero
    if not isinstance(año, int):
        return "El año debe ser un número entero."

    # Verificar que el año ingresado esté en la columna 'posted_year'
    if año not in df_f3_4_5['posted_year'].unique():
        return "El año no se encuentra en la columna 'posted_year'."

    # Filtrar el dataset para obtener solo las filas correspondientes al año dado
    juegos_del_año = df_f3_4_5[df_f3_4_5['posted_year'] == año]

    # Calcular la cantidad de recomendaciones para cada developer
    recomendaciones_por_juego = juegos_del_año.groupby('developer')['recommend'].sum().reset_index()

    # Ordenar los juegos por la cantidad de recomendaciones en orden descendente
    devs_ordenados = recomendaciones_por_juego.sort_values(by='recommend', ascending=True)

    # Tomar los tres primeros lugares
    ultimo_puesto = devs_ordenados.iloc[0]['developer']
    penultimo_puesto = devs_ordenados.iloc[1]['developer']
    antepenultimo_puesto = devs_ordenados.iloc[2]['developer']

    # Crear el diccionario con los tres primeros lugares
    ultimos_tres = {
        "Puesto 1": ultimo_puesto,
        "Puesto 2": penultimo_puesto,
        "Puesto 3": antepenultimo_puesto
    }

    return ultimos_tres

In [28]:
UsersWorstDeveloper(2015)

{'Puesto 1': 'Bitl,Cobalt-57',
 'Puesto 2': 'Lazy Bum Games',
 'Puesto 3': 'Sometimes You'}

## Quinta función
- def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor. <br>
Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [29]:
def sentiment_analysis(devs:str):

    # Filtrar el DataFrame por la desarrolladora ingresada
    df_devs = df_f3_4_5[df_f3_4_5['developer'] == devs]

    # Verificar si se encontraron registros para el año
    if df_devs.empty:
        return {"Mensaje": "No se encontraron registros para la desarrolladora especificada."}

    # Contar la cantidad de registros para cada categoría de análisis de sentimiento
    sentiment_analysis_column = df_devs['sentiment_analysis']
    sentiment = sentiment_analysis_column.value_counts().to_dict()

    # Crear una lista con los resultados, tuve que colocar en formato de str todo para que me lo tomara como válido 
    resultado= list(["Negative= "+ str(sentiment.get(0, 0)), "Neutral= "+ str(sentiment.get(1, 0)), "Positive= "+ str(sentiment.get(2, 0))])

    #Crear el diccionario final
    result = {
        devs:resultado
    }

    return result

In [30]:
sentiment_analysis('Valve')

{'Valve': ['Negative= 929', 'Neutral= 2049', 'Positive= 5393']}

Creo un dataset exclusivo para machine learning

In [241]:
df_ML= pd.merge(df_f1_2,df_f3_4_5,on='item_id')

In [245]:
#df_ML.drop(columns='posted_year',inplace=True)
df_ML.drop(columns='user_id_y',inplace=True)
df_ML.drop(columns='developer_y',inplace=True)
df_ML.drop(columns='app_name_y',inplace=True)
df_ML.drop(columns='genres_y',inplace=True)
df_ML.drop(columns='item_name',inplace=True)
df_ML.drop(columns='release_date',inplace=True)
df_ML.drop(columns='title',inplace=True)
df_ML.drop(columns='release_year',inplace=True)

In [249]:
df_ML = df_ML.rename(columns={"genres_x": "genres"})
df_ML = df_ML.rename(columns={"app_name_x": "app_name"})
df_ML = df_ML.rename(columns={"developer_x": "developer"})
df_ML = df_ML.rename(columns={"user_id_x": "user_id"})

In [253]:
#Guardo el dataset en un archivo csv
#df_ML.to_csv('df_ML.csv')

### EDA <br>

- Se buscan outliers en variables de cantidad, solo se encuentra en 'playtime_forever' usando .describe() y se eliminan los mayores a 6500 horas de juego ya que es muy poco probable que alguien dedique tantas horas de juego diarias a lo largo de 5 años

In [254]:
df_eda=pd.read_csv('df_ML.csv')

In [268]:
#df_eda.drop(columns='Unnamed: 0',inplace=True)

In [263]:
df_eda.describe()

,item_id,sentiment_analysis,playtime_forever
count,33333.000000,33333.000000,33333.000000
mean,166942.891729,1.476075,2104.298593
std,132670.561269,0.754684,9856.081973
min,10.000000,0.000000,1.000000
25%,6830.000000,1.000000,46.000000
50%,212070.000000,2.000000,225.000000
75%,261880.000000,2.000000,919.000000
max,527340.000000,2.000000,400827.000000


In [266]:

df_sin_outliers_2= df_eda[df_eda['playtime_forever']<6500.0]

In [269]:
df_sin_outliers=df_sin_outliers_2

In [271]:
#Guardo el dataset con el trabajo de EDA

df_sin_outliers.to_csv('df_EDA.csv')